## Setup

In [2]:
%matplotlib inline

import warnings
warnings.simplefilter(action = "ignore", category = RuntimeWarning)

# External modules
import ast
import numpy as np
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
import os
from pathlib import Path

In [3]:
# Path management
main_dir = str(Path(os.path.abspath('')).parents[0])
os.chdir(main_dir)
print('main dir:',main_dir)

main dir: /Users/pablo/OneDrive/data-science/github/isuelogit


In [4]:
# Internal modules
from src import isuelogit as isl

In [5]:
# Seed
_SEED = 2022

## Read network data from Fresno

In [6]:
network_name = 'Fresno'

# To write estimation report and set seed for all algorithms involving some randomness  
estimation_reporter = isl.writer.Reporter(
    folderpath = isl.config.dirs['output_folder'] + 'estimations/' + network_name, seed = _SEED)

# Reader of geospatial and spatio-temporal data
data_reader = isl.etl.DataReader(network_key=network_name)

# First Tuesday of October, 2019
data_reader.select_period(date='2019-10-01', hour=16)

# Read nodes data
nodes_df = pd.read_csv(isl.dirs['input_folder'] + '/network-data/nodes/'  + 'fresno-nodes-data.csv')

# Read spatiotemporal link data
links_df = pd.read_csv(
    isl.dirs['input_folder'] + 'network-data/links/' + str(data_reader.options['selected_date'])+ '-fresno-link-data.csv',
    converters={"link_key": ast.literal_eval,"pems_id": ast.literal_eval})


Selected date is 2019-10-01, Tuesday at 16:00


## Build network

In [7]:
network_generator = isl.factory.NetworkGenerator()

A = network_generator.generate_adjacency_matrix(links_keys=list(links_df['link_key'].values))

fresno_network = \
    network_generator.build_fresno_network(A=A, links_df=links_df, nodes_df=nodes_df, network_name= network_name)


Creating Fresno network

Nodes: 1789, Links: 2413


### OD matrix

In [8]:
network_generator.read_OD(network=fresno_network, sparse=True)

Matrix Q (1789, 1789) read in 0.0[s] with sparse format
66266.3 trips were loaded among 6970 o-d pairs


### Paths

In [9]:
paths_generator = isl.factory.PathsGenerator()

# # Generate and Load paths in network
# paths_generator.load_k_shortest_paths(network = fresno_network, k=3)

# Read and load paths in network
paths_generator.read_paths(network=fresno_network, update_incidence_matrices=True)

Reading paths: |####----------------| 21.6% 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



13905 paths were read in 16.6[s]              

13905 paths were loaded in the network

Updating incidence matrices

Matrix D (2413, 13905) generated in 35.4[s] 

Matrix M (6970, 13905) generated in 12.7[s] 

Matrix C (13905, 13905) generated in 6.7[s]



### Link performance functions

In [10]:
bpr_parameters_df = pd.DataFrame({'link_key': links_df['link_key'],
                                  'alpha': links_df['alpha'],
                                  'beta': links_df['beta'],
                                  'tf': links_df['tf_inrix'],
                                  #'tf': links_df['tf'],
                                  'k': pd.to_numeric(links_df['k'], errors='coerce', downcast='float')
                                  })

# Normalize free flow travel time between 0 and 1
bpr_parameters_df['tf'] = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(np.array(bpr_parameters_df['tf']).reshape(-1, 1)))

fresno_network.set_bpr_functions(bprdata=bpr_parameters_df)

### Feature Engineering

In [11]:
fresno_network.load_features_data(links_df, link_key = 'link_key')

isl.etl.feature_engineering_fresno(links=fresno_network.links, network=fresno_network)

Features values of links with a type different than LWRLK were set to 0
New features: ['low_inc', 'high_inc', 'no_incidents', 'no_bus_stops', 'no_intersections', 'tt_sd_adj', 'tt_reliability']


In [12]:
features_list = ['median_inc', 'intersections', 'incidents', 'bus_stops', 'median_age',
                 'tt_avg', 'tt_sd','tt_var', 'tt_cv', 'tt_sd_adj','tt_reliability',
                 'speed_ref_avg', 'speed_avg', 'speed_hist_avg','speed_sd','speed_hist_sd','speed_cv']


# Normalization of features to range [0,1]
linkdata = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(fresno_network.Z_data[features_list].values))
linkdata.columns = features_list
linkdata.insert(0, 'link_key', fresno_network.links_keys)

fresno_network.load_features_data(linkdata)

### Traffic counts

In [13]:
counts_df = pd.read_csv(isl.dirs['input_folder'] + '/network-data/links/' \
                            + str(data_reader.options['selected_date']) + '-fresno-link-counts' + '.csv',
                        converters={'link_key': ast.literal_eval})

counts = dict(zip(counts_df['link_key'].values, counts_df['counts'].values))

fresno_network.load_traffic_counts(counts=counts)

## Descriptive statistics

### Link coverage

In [14]:
total_counts_observations = np.count_nonzero(~np.isnan(np.array(list(counts.values()))))

total_links = np.array(list(counts.values())).shape[0]

print('\nTotal link counts observations: ' + str(total_counts_observations))
print('Link coverage: ' + "{:.1%}".format(round(total_counts_observations / total_links, 4)))


Total link counts observations: 141
Link coverage: 5.8%


### Networks topology

In [15]:
isl.descriptive_statistics.summary_table_networks([fresno_network])

,network,nodes,links,ods,paths
0,Fresno,1789,2413,6970,13905


### Links performance functions

In [16]:
bpr_parameters_df.describe()

,alpha,beta,tf,k
count,2.413000e+03,2413.0,2413.000000,2.413000e+03
mean,1.500000e-01,4.0,0.075482,1.439305e+09
std,2.776133e-17,0.0,0.085996,2.261116e+09
min,1.500000e-01,4.0,0.000000,1.800000e+03
25%,1.500000e-01,4.0,0.000000,1.800000e+03
50%,1.500000e-01,4.0,0.057738,1.800000e+03
75%,1.500000e-01,4.0,0.106484,4.990000e+09
max,1.500000e-01,4.0,1.000000,4.990000e+09


### Links features

In [17]:
summary_table_links_df = fresno_network.Z_data
estimation_reporter.write_table(df = summary_table_links_df, filename = 'links_data.csv', float_format = '%.3f')
summary_table_links_df

,link_type,alpha,beta,tf,k,observed,counts,capacity [veh],tt_ff [min],speed_ff[mi/hr],...,bus_stops,intersections,tf_inrix,high_inc,low_inc,no_incidents,no_bus_stops,no_intersections,tt_sd_adj,tt_reliability
0,LWRLK,0.15,4.0,0.098,1.800000e+03,0,NaN,1800.0,0.098,45,...,0.0,0.111111,0.221,1,0,1,1,0,0.109299,0.462004
1,LWRLK,0.15,4.0,0.169,1.800000e+03,0,NaN,1800.0,0.169,50,...,0.0,0.000000,0.203,1,0,0,1,1,0.071056,0.609407
2,LWRLK,0.15,4.0,0.396,2.400000e+03,0,NaN,2400.0,0.396,65,...,0.0,0.000000,0.396,1,0,0,1,1,0.000000,0.000000
3,LWRLK,0.15,4.0,0.192,2.000000e+03,0,NaN,2000.0,0.192,25,...,0.0,0.111111,0.140,1,0,1,1,0,0.017797,0.701906
4,LWRLK,0.15,4.0,0.105,1.800000e+03,0,NaN,1800.0,0.105,35,...,0.0,0.111111,0.105,1,0,1,1,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2408,PQULK,0.15,4.0,0.000,4.990000e+09,0,NaN,inf,0.000,99999,...,0.0,0.000000,0.000,0,0,0,0,0,0.000000,0.000000
2409,PQULK,0.15,4.0,0.000,4.990000e+09,0,NaN,inf,0.000,99999,...,0.0,0.000000,0.000,0,0,0,0,0,0.000000,0.000000
2410,LWRLK,0.15,4.0,0.045,1.800000e+03,0,NaN,1800.0,0.045,65,...,0.0,0.111111,0.040,1,0,1,1,0,0.005085,0.825842
2411,LWRLK,0.15,4.0,0.107,2.400000e+03,0,NaN,2400.0,0.107,65,...,0.0,0.222222,0.099,0,1,0,1,0,0.004827,0.798930


In [18]:
summary_table_links_df.describe()

,alpha,beta,tf,k,observed,counts,capacity [veh],tt_ff [min],speed_ff[mi/hr],inrix_id,...,bus_stops,intersections,tf_inrix,high_inc,low_inc,no_incidents,no_bus_stops,no_intersections,tt_sd_adj,tt_reliability
count,2.413000e+03,2413.0,2413.000000,2.413000e+03,2413.000000,141.000000,2413.0,2413.000000,2413.000000,1.468000e+03,...,2413.000000,2413.000000,2413.000000,2413.000000,2413.000000,2413.000000,2413.000000,2413.000000,2413.000000,2413.000000
mean,1.500000e-01,4.0,0.150823,1.439305e+09,0.058433,2213.607801,inf,0.150823,28872.664318,9.131292e+08,...,0.037505,0.097389,0.159494,0.525073,0.186490,0.639867,0.587236,0.239536,0.046804,0.399915
std,2.776133e-17,0.0,0.172458,2.261116e+09,0.234610,849.965115,NaN,0.172458,45293.922006,6.019207e+08,...,0.110321,0.146641,0.181710,0.499474,0.389582,0.480138,0.492433,0.426889,0.071903,0.351537
min,1.500000e-01,4.0,0.000000,1.800000e+03,0.000000,111.000000,1800.0,0.000000,15.000000,1.685461e+08,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.500000e-01,4.0,0.000000,1.800000e+03,0.000000,1698.000000,1800.0,0.000000,40.000000,4.416712e+08,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.500000e-01,4.0,0.121000,1.800000e+03,0.000000,2101.000000,1800.0,0.121000,45.000000,4.498919e+08,...,0.000000,0.000000,0.122000,1.000000,0.000000,1.000000,1.000000,0.000000,0.017366,0.549181
75%,1.500000e-01,4.0,0.205000,4.990000e+09,0.000000,2717.000000,NaN,0.205000,99999.000000,1.626675e+09,...,0.000000,0.111111,0.225000,1.000000,0.000000,1.000000,1.000000,0.000000,0.071056,0.723356
max,1.500000e-01,4.0,2.113000,4.990000e+09,1.000000,4807.000000,inf,2.113000,99999.000000,1.626774e+09,...,1.000000,1.000000,2.113000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


#### Travel demand

In [19]:
# Cumulative trips OD matrix
# isl.visualization.Artist().cumulative_demand(Q= fresno_network.Q,
#                          folderpath = estimation_reporter.dirs['estimation_folder'],
#                          filename = 'cumulative_OD_matrix.pdf')

## Bilevel Estimation

In [20]:
utility_function = isl.estimation.UtilityFunction(
    features_Y=['tt'],
    initial_values={'tt': 0},
)

equilibrator_norefined = isl.equilibrium.LUE_Equilibrator(
    network=fresno_network,
    paths_generator=paths_generator,
    utility_function=utility_function,
    max_iters=100,
    method='fw',
    iters_fw=10,
    column_generation={'n_paths': 18, 'ods_coverage': 0.25, 'paths_selection': 10, 'ods_sampling': 'demand_sequentially'},
    path_size_correction=1
)

outer_optimizer_norefined = isl.estimation.OuterOptimizer(
    method='ngd',
    iters=1,
    eta=2e-1,
)

learner_norefined = isl.estimation.Learner(
    equilibrator=equilibrator_norefined,
    outer_optimizer=outer_optimizer_norefined,
    utility_function=utility_function,
    network=fresno_network,
    name = 'norefined'
)

equilibrator_refined = isl.equilibrium.LUE_Equilibrator(
    network=fresno_network,
    paths_generator=paths_generator,
    utility_function=utility_function,
    max_iters=100,
    method='fw',
    iters_fw=10,
    # column_generation={'n_paths': 12, 'ods_coverage': 0.1, 'paths_selection': None},
    path_size_correction=1
)

outer_optimizer_refined = isl.estimation.OuterOptimizer(
    method='lm',
    # method='ngd',
    iters=1,
    # eta=1e-1,
)

learner_refined = isl.estimation.Learner(
    network=fresno_network,
    equilibrator=equilibrator_refined,
    outer_optimizer=outer_optimizer_refined,
    utility_function=utility_function,
    name = 'refined'
)

### a) Benchmark predictions

#### Naive prediction using mean counts

In [21]:
mean_counts_prediction_loss, mean_count_benchmark_model \
    = isl.estimation.mean_count_prediction(counts=np.array(list(counts.values()))[:, np.newaxis])

print('\nObjective function under mean count prediction: ' + '{:,}'.format(round(mean_counts_prediction_loss, 1)))


Objective function under mean count prediction: 101,141,667.7


#### Naive prediction where travelers make equilikely choices

In [22]:
equilikely_prediction_loss, predicted_counts_equilikely \
    = isl.estimation.loss_counts_equilikely_choices(
    network = fresno_network,
    equilibrator=equilibrator_norefined,
    counts=fresno_network.observed_counts_vector,
    utility_function=utility_function)

print('Objective function under equilikely route choices: ' + '{:,}'.format(round(equilikely_prediction_loss, 1)))

Objective function under equilikely route choices: 279,378,072.3


### b) Model with travel time feature only

#### No refined stage

In [ ]:
learning_results_norefined_traveltime_model, inference_results_norefined_traveltime_model, best_iter_norefined_traveltime_model = \
    learner_norefined.statistical_inference(h0=0, bilevel_iters=10, alpha=0.05, link_report=False, iteration_report = False)

theta_norefined_traveltime_model = learning_results_norefined_traveltime_model[best_iter_norefined_traveltime_model]['theta']

paths_generator.write_paths(network=fresno_network, overwrite_input=False, filename = 'paths-traveltime-model-Fresno.csv')


Bilevel optimization for Fresno network 

Initial theta: {'tt': '0.0E+00'}


#### Refined stage

In [ ]:
learner_refined.utility_function.initial_values = theta_norefined_traveltime_model

learning_results_refined_traveltime_model, inference_results_refined_traveltime_model, best_iter_refined_traveltime_model = \
    learner_refined.statistical_inference(h0=0, bilevel_iters=10, alpha=0.05, link_report=False, iteration_report = False)

theta_refined_traveltime_model = learning_results_refined_traveltime_model[best_iter_refined_traveltime_model]['theta']

#### Report

In [ ]:
estimation_reporter.add_items_report(
    selected_date = data_reader.options['selected_date'],
    selected_hour = data_reader.options['selected_hour'],
    selected_od_periods = data_reader.options['od_periods'],
    mean_counts=round(mean_count_benchmark_model,1),
    mean_counts_prediction_loss = round(mean_counts_prediction_loss,1),
    equilikely_prediction_loss = round(equilikely_prediction_loss,1)
)

estimation_reporter.add_items_report(
    theta_norefined=theta_norefined_traveltime_model,
    theta_refined= theta_refined_traveltime_model,
    best_objective_norefined = round(learning_results_norefined_traveltime_model[best_iter_norefined_traveltime_model]['objective'],1),
    best_objective_refined = round(learning_results_refined_traveltime_model[best_iter_refined_traveltime_model]['objective'],1),
)

# Summary with most relevant options, prediction error, initial parameters, etc
estimation_reporter.write_estimation_report(
    network=fresno_network,
    learners=[learner_norefined, learner_refined],
    utility_function=utility_function)

# Write tables with results on learning and inference
estimation_reporter.write_learning_tables(
    results_norefined=learning_results_norefined_traveltime_model,
    results_refined=learning_results_refined_traveltime_model,
    network = fresno_network,
    utility_function = utility_function)

estimation_reporter.write_inference_tables(
    results_norefined=inference_results_norefined_traveltime_model,
    results_refined=inference_results_refined_traveltime_model,
    float_format = '%.3f')

#### Visualizations

##### Convergence

In [ ]:
results_traveltime_model_df = isl.descriptive_statistics \
    .get_loss_and_estimates_over_iterations(results_norefined=learning_results_norefined_traveltime_model
                                            , results_refined=learning_results_refined_traveltime_model)

fig = isl.visualization.Artist().convergence(
    results_norefined_df=results_traveltime_model_df[results_traveltime_model_df['stage'] == 'norefined'],
    results_refined_df=results_traveltime_model_df[results_traveltime_model_df['stage'] == 'refined'],
    filename='convergence_' + fresno_network.key,
    methods=[outer_optimizer_norefined.method.key, outer_optimizer_refined.method.key],
    folder = estimation_reporter.dirs['estimation_folder'],
    simulated_data = False
)

##### Distribution of errors across link counts

In [ ]:
best_predicted_counts_norefined = np.array(list(learning_results_norefined_traveltime_model[best_iter_norefined_traveltime_model]['x'].values()))[:, np.newaxis]
best_predicted_counts_refined = np.array(list(learning_results_refined_traveltime_model[best_iter_refined_traveltime_model]['x'].values()))[:, np.newaxis]

fig, axs = plt.subplots(1, 2, sharey='all', tight_layout=True, figsize=(8, 4))

axs[0].hist(isl.estimation.error_by_link(observed_counts=np.array(list(counts.values()))[:, np.newaxis],
                                         predicted_counts=best_predicted_counts_norefined))
axs[1].hist(isl.estimation.error_by_link(observed_counts=np.array(list(counts.values()))[:, np.newaxis],
                                         predicted_counts=best_predicted_counts_refined))

for axi in [axs[0], axs[1]]:
    axi.tick_params(axis='x', labelsize=16)
    axi.tick_params(axis='y', labelsize=16)

plt.show()

fig.savefig(estimation_reporter.dirs['estimation_folder'] + '/' + 'distribution_predicted_error_counts.pdf',
            pad_inches=0.1, bbox_inches="tight")

### c) Model with all features

In [ ]:
utility_function_full_model = isl.estimation.UtilityFunction(
    features_Y=['tt'],
#    features_Z= ['incidents'],
    features_Z= ['incidents', 'tt_cv','median_inc', 'bus_stops', 'intersections'],
    # initial_values = {'median_inc': 1.5, 'bus_stops':0},
    signs = {'tt': '-', 'tt_cv':'-', 'incidents':'-', 'intersections':'-', 'bus_stops':'-', 'median_inc':'+'},
    fixed = {'tt': True}
)

# Update utility functions of no refined and refined learners
learner_norefined.utility_function = utility_function_full_model
learner_refined.utility_function = utility_function_full_model

#Initialize value with the estimate obtained from b)
learner_norefined.utility_function.initial_values = theta_refined_traveltime_model
# learner_norefined.utility_function.zero_initializer()

learner_norefined.outer_optimizer.method.eta = 5e-1
learner_refined.outer_optimizer.method.eta = 5e-1

paths_generator.read_paths(network=fresno_network, update_incidence_matrices=True,
                           filename = 'paths-traveltime-model-Fresno.csv',
                           folderpath = isl.config.dirs['write_network_data'] + 'paths/'
                           )

#### No refined stage

In [ ]:
learning_results_norefined_full_model, inference_results_norefined_full_model, best_iter_norefined_full_model = \
    learner_norefined.statistical_inference(h0=0, bilevel_iters=10, alpha=0.05, link_report=False, iteration_report = True,
                                            parameters_constraints = {'fixed': True, 'sign': True})

theta_norefined_full_model = learning_results_norefined_full_model[best_iter_norefined_full_model]['theta']

paths_generator.write_paths(network=fresno_network, overwrite_input=False, filename = 'paths-full-model-Fresno.csv')

#### Refined stage

In [ ]:
features_Y,features_Z = isl.estimation.feature_selection(utility_function_full_model,
                                                          theta = theta_norefined_full_model,
                                                          criterion = 'sign')

utility_function_full_model = isl.estimation.UtilityFunction(
    features_Y = features_Y,
    features_Z = features_Z,
    fixed = {'tt': True}
)

learner_norefined.utility_function = utility_function_full_model
learner_refined.utility_function = utility_function_full_model

learner_refined.utility_function.initial_values = theta_norefined_full_model

learning_results_refined_full_model, inference_results_refined_full_model, best_iter_refined_full_model = \
    learner_refined.statistical_inference(h0=0, bilevel_iters=10, alpha=0.05, link_report=False, iteration_report = True,
                                            parameters_constraints = {'fixed': False, 'sign': False})

theta_refined_full_model = learning_results_refined_full_model[best_iter_refined_full_model]['theta']

#### Report

In [ ]:
estimation_reporter.add_items_report(
    selected_date = data_reader.options['selected_date'],
    selected_hour = data_reader.options['selected_hour'],
    selected_od_periods = data_reader.options['od_periods'],
    mean_counts=round(mean_count_benchmark_model,1),
    mean_counts_prediction_loss = round(mean_counts_prediction_loss,1),
    equilikely_prediction_loss = round(equilikely_prediction_loss,1)
)

estimation_reporter.add_items_report(
    theta_norefined=theta_norefined_full_model,
    theta_refined= theta_refined_full_model,
    best_objective_norefined = round(learning_results_norefined_full_model[best_iter_norefined_full_model]['objective'],1),
    best_objective_refined = round(learning_results_refined_full_model[best_iter_refined_full_model]['objective'],1),
)

estimation_reporter.write_estimation_report(
    network=fresno_network,
    learners=[learner_norefined, learner_refined],
    utility_function=utility_function_full_model)

estimation_reporter.write_learning_tables(
    results_norefined=learning_results_norefined_full_model,
    results_refined=learning_results_refined_full_model,
    network = fresno_network,
    utility_function = utility_function_full_model)

estimation_reporter.write_inference_tables(
    results_norefined=inference_results_norefined_full_model,
    results_refined=inference_results_refined_full_model,
    float_format = '%.3f')

#### Visualization

In [ ]:
results_full_model_df = isl.descriptive_statistics \
    .get_loss_and_estimates_over_iterations(results_norefined=learning_results_norefined_full_model,
                                            results_refined=learning_results_refined_full_model)


# Convergence with curves for each parameter and comparing the travel time and full models
fig = isl.visualization.Artist().convergence_models(
    results_dfs={'traveltime_model': results_traveltime_model_df, 'full_model': results_full_model_df},
    features= {'tt': 'travel time', 'tt_cv':'coeficient of variation of travel time', 'incidents':'incidents', 'intersections':'intersections', 'bus_stops':'bus_stops', 'median_inc':'median income area'},
    filename='convergence_traveltime_and_full_models_' + fresno_network.key,
    folder=estimation_reporter.dirs['estimation_folder']
)

### d) Model with feature engineering

In [ ]:
utility_function_feature_engineering_model = isl.estimation.UtilityFunction(
    features_Y=['tt'],
    features_Z= ['tt_sd_asj', 'no_incidents', 'no_bus_stops', 'no_intersections', 'low_inc'],
    signs = {'tt': '-', 'tt_sd_adj':'-', 'no_incidents':'+', 'no_bus_stops':'+', 'no_intersections':'+', 'low_inc':'-'},
    fixed = {'tt': True}
)

# Update utility functions of no refined and refined learners
learner_norefined.utility_function = utility_function_feature_engineering_model
learner_refined.utility_function = utility_function_feature_engineering_model

#Initialize travel time estimate with the one obtained from b) and load those paths as well
learner_norefined.utility_function.initial_values = theta_refined_traveltime_model

paths_generator.read_paths(network=fresno_network, update_incidence_matrices=True,
                           filename = 'paths-traveltime-model-Fresno.csv',
                           folderpath = isl.config.dirs['write_network_data'] + 'paths/'
                           )

learner_norefined.outer_optimizer.method.eta = 5e-1
learner_refined.outer_optimizer.method.eta = 5e-1

estimation_reporter = isl.writer.Reporter(folderpath = isl.config.dirs['output_folder'] + 'estimations/' + network_name, seed =_SEED)

#### No refined stage

In [ ]:
learning_results_norefined_feature_engineering_model, inference_results_norefined_feature_engineering_model, best_iter_norefined_feature_engineering_model = \
    learner_norefined.statistical_inference(h0=0, bilevel_iters=10, alpha=0.05, link_report=False, iteration_report = True,
                            parameters_constraints = {'fixed': True, 'sign': True})

theta_norefined_feature_engineering_model = learning_results_norefined_feature_engineering_model[best_iter_norefined_feature_engineering_model]['theta']

paths_generator.write_paths(network=fresno_network, overwrite_input=False, filename = 'paths-feature-engineering-model-Fresno.csv')

#### Refined stage

In [ ]:
features_Y,features_Z = isl.estimation.feature_selection(utility_function_feature_engineering_model,
                                                          theta = theta_norefined_feature_engineering_model,
                                                          criterion = 'sign')

utility_function_feature_engineering_model = isl.estimation.UtilityFunction(
    features_Y = features_Y,
    features_Z = features_Z
)

learner_norefined.utility_function = utility_function_feature_engineering_model
learner_refined.utility_function = utility_function_feature_engineering_model

learner_refined.utility_function.initial_values = theta_norefined_feature_engineering_model

learning_results_refined_feature_engineering_model, inference_results_refined_feature_engineering_model, best_iter_refined_feature_engineering_model = \
    learner_refined.statistical_inference(h0=0, bilevel_iters=10, alpha=0.05, link_report=False, iteration_report = True,
                                          parameters_constraints = {'fixed': False, 'sign': False})

theta_refined_feature_engineering_model = learning_results_refined_feature_engineering_model[best_iter_refined_feature_engineering_model]['theta']

#### Report

In [ ]:
estimation_reporter.add_items_report(
    selected_date = data_reader.options['selected_date'],
    selected_hour = data_reader.options['selected_hour'],
    selected_od_periods = data_reader.options['od_periods'],
    mean_counts=round(mean_count_benchmark_model,1),
    mean_counts_prediction_loss = round(mean_counts_prediction_loss,1),
    equilikely_prediction_loss = round(equilikely_prediction_loss,1)
)

estimation_reporter.add_items_report(
    theta_norefined=theta_norefined_feature_engineering_model,
    theta_refined= theta_refined_feature_engineering_model,
    best_objective_norefined = round(learning_results_norefined_feature_engineering_model[best_iter_norefined_feature_engineering_model]['objective'],1),
    best_objective_refined = round(learning_results_refined_feature_engineering_model[best_iter_refined_feature_engineering_model]['objective'],1),
)

estimation_reporter.write_estimation_report(
    network=fresno_network,
    learners=[learner_norefined, learner_refined],
    utility_function=utility_function_feature_engineering_model)

estimation_reporter.write_learning_tables(
    results_norefined=learning_results_norefined_feature_engineering_model,
    results_refined=learning_results_refined_feature_engineering_model,
    network = fresno_network,
    utility_function = utility_function_feature_engineering_model)

estimation_reporter.write_inference_tables(
    results_norefined=inference_results_norefined_feature_engineering_model,
    results_refined=inference_results_refined_feature_engineering_model,
    float_format = '%.3f')

#### Visualization

In [ ]:
results_feature_engineering_model_df = isl.descriptive_statistics \
    .get_loss_and_estimates_over_iterations(results_norefined=learning_results_norefined_feature_engineering_model,
                                            results_refined=learning_results_refined_feature_engineering_model)


# Convergence with curves for each parameter and comparing the travel time and full models
fig = isl.visualization.Artist().convergence_models(
    results_dfs={'traveltime_model': results_traveltime_model_df, 'full_model': results_feature_engineering_model_df},
    features= {'tt': 'travel time', 'tt_sd_adj':'adjusted std of travel time', 'no_incidents':'no incidents', 'no_intersections':'no intersections', 'no_bus_stops':'no bus_stops', 'low_inc':'low income area'},
    filename='convergence_traveltime_and_full_models_' + fresno_network.key,
    folder=estimation_reporter.dirs['estimation_folder']
)